In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.SILENT)

In [2]:
print('This example illustrates a layered network with a loop.')

model = LayeredNetwork('myLayeredModel')

# Layer 1 - Reference task
P1 = Processor(model, 'P1', GlobalConstants.MaxInt, SchedStrategy.INF)
T1 = Task(model, 'T1', 1, SchedStrategy.REF).on(P1)
T1.setThinkTime(Immediate())
E1 = Entry(model, 'Entry').on(T1)

# Layer 2 - Service task
P2 = Processor(model, 'P2', GlobalConstants.MaxInt, SchedStrategy.INF)
T2 = Task(model, 'T2', 1, SchedStrategy.INF).on(P2).setThinkTime(Immediate())
E2 = Entry(model, 'E2').on(T2)

# Layer 3 - Backend task
P3 = Processor(model, 'P3', 5, SchedStrategy.PS)
T3 = Task(model, 'T3', 20, SchedStrategy.INF).on(P3)
T3.setThinkTime(Exp.fitMean(10))
E3 = Entry(model, 'E1').on(T3)  # Note: Entry named 'E1' in original

# Activities for T1 (with loop)
A1 = Activity(model, 'A1', Exp.fitMean(1)).on(T1).boundTo(E1)
A2 = Activity(model, 'A2', Exp.fitMean(2)).on(T1)
A3 = Activity(model, 'A3', Exp.fitMean(3)).on(T1).synchCall(E2, 1)

# Activities for T2 (with fork-join)
B1 = Activity(model, 'B1', Exp.fitMean(0.1)).on(T2).boundTo(E2)
B2 = Activity(model, 'B2', Exp.fitMean(0.2)).on(T2)
B3 = Activity(model, 'B3', Exp.fitMean(0.3)).on(T2)
B4 = Activity(model, 'B4', Exp.fitMean(0.4)).on(T2)
B5 = Activity(model, 'B5', Exp.fitMean(0.5)).on(T2)
B6 = Activity(model, 'B6', Exp.fitMean(0.6)).on(T2).synchCall(E3, 1).repliesTo(E2)

# Activities for T3 (with or-fork/join)
C1 = Activity(model, 'C1', Exp.fitMean(0.1)).on(T3).boundTo(E3)
C2 = Activity(model, 'C2', Exp.fitMean(0.2)).on(T3)
C3 = Activity(model, 'C3', Exp.fitMean(0.3)).on(T3)
C4 = Activity(model, 'C4', Exp.fitMean(0.4)).on(T3)
C5 = Activity(model, 'C5', Exp.fitMean(0.5)).on(T3).repliesTo(E3)

This example illustrates a layered network with a loop.


In [3]:
# Add precedence relationships

# T1: Loop with 3 iterations
T1.addPrecedence(ActivityPrecedence.Loop(A1, [A2, A3], 3))

# T2: Serial connection and AND fork-join
T2.addPrecedence(ActivityPrecedence.Serial(B4, B5))
T2.addPrecedence(ActivityPrecedence.AndFork(B1, [B2, B3, B4]))
T2.addPrecedence(ActivityPrecedence.AndJoin([B2, B3, B5], B6))

# T3: OR fork-join with probabilities
T3.addPrecedence(ActivityPrecedence.OrFork(C1, [C2, C3, C4], [0.3, 0.3, 0.4]))
T3.addPrecedence(ActivityPrecedence.OrJoin([C2, C3, C4], C5))

In [4]:
# Solve with different solvers

if SolverLQNS.isAvailable():
    solver_lqns = SolverLQNS(model)
    avg_table_lqns = solver_lqns.get_avg_table()

     Node   NodeType    QLen    Util    RespT  ResidT  ArvR    Tput
0      P1  Processor     NaN  0.7958      NaN     0.0   0.0     NaN
1      P2  Processor     NaN  0.1671      NaN     0.0   0.0     NaN
2      P3  Processor     NaN  0.0724      NaN     0.0   0.0     NaN
3      T1    RefTask  1.0000  0.7958      NaN     0.0   0.0  0.0796
4      T2       Task  0.2042  0.1671      NaN     0.0   0.0  0.0796
5      T3       Task  0.0724  0.0724      NaN     0.0   0.0  0.0796
6   Entry      Entry  1.0000  0.7958  12.5667     0.0   0.0  0.0796
7      E2      Entry  0.2042  0.1671   2.5667     0.0   0.0  0.0796
8      E1      Entry  0.0724  0.0724   0.9098     0.0   0.0  0.0796
9      A1   Activity  0.0796  0.0796   1.0000     0.0   0.0  0.0796
10     A2   Activity  0.4775  0.4775   2.0000     0.0   0.0  0.2387
11     A3   Activity  0.4430  0.2387   5.5667     0.0   0.0  0.0796
12     B1   Activity  0.0080  0.0080   0.1000     0.0   0.0  0.0796
13     B2   Activity  0.0159  0.0159   0.2000   

In [5]:
solver_ln = SolverLN(model)
avg_table_ln = solver_ln.get_avg_table()

SolverLN initialization completed. Starting iteration on ensemble models.
Iter 1.Analyze time: 0.593035s. Update time: 0.001328s. Runtime: 0.606098s.
Iter 2.Analyze time: 0.509590s. Update time: 0.000264s. Runtime: 1.117572s.
Iter 3.Analyze time: 0.530334s. Update time: 0.000545s. Runtime: 1.648403s.
Iter 4.Analyze time: 0.502539s. Update time: 0.000319s. Runtime: 2.151876s.
Iter 5.Analyze time: 0.506623s. Update time: 0.000348s. Runtime: 2.659189s.
Iter 6.Analyze time: 0.476367s. Update time: 0.000363s. Runtime: 3.136123sTesting convergence.

Iter 7.Analyze time: 0.533111s. Update time: 0.000248s. Runtime: 3.670100s.
SolverLN completed in 5 iterations. Running final checks.\n
Summary: Analyze avg time: 0.521657s. Update avg time: 0.000488s. Total runtime: 4.140292s. 
     Node   NodeType    QLen    Util    RespT   ResidT  ArvR    Tput
0      P1  Processor     NaN  0.8280      NaN      NaN   NaN     NaN
1      P2  Processor     NaN  0.1716      NaN      NaN   NaN     NaN
2      P3  Pro